<div style='background-color: orange'>
<a id="TableOfContents"></a>
    <h1 style='text-align: center'>
        <b><i>
            TABLE OF CONTENTS:
        </i></b></h1>
<li><a href='#imports'>Imports</a></li>
<li><a href="#acquire">Acquire</a></li>
<li><a href='#prepare'>Prepare</a></li>
<li><a href="#wrangle">Wrangle</a></li>
<li><a href='#misc'>Miscellaneous</a></li>

<div style='background-color: orange'>
<a id="imports"></a>
    <h1 style='text-align: center'>
        <b><i>
            Imports
        </i></b></h1>
<li><a href='#TableOfContents'>Table of Contents</a></li>

In [1]:
# Vectorization and tables
import numpy as np
import pandas as pd

# .py files
import wrangle as w

<div style='background-color: orange'>
<a id="acquire"></a>
    <h1 style='text-align: center'>
        <b><i>
            Acquire
        </i></b></h1>
<li><a href='#TableOfContents'>Table of Contents</a></li>

Acquire everything from the vanilla mass_shooters database via excel sheet

- mass_shooters Vanilla Shape:
    - Rows: 189
    - Columns: 153

In [2]:
# Acquire the vanilla data
mass_shooters = pd.read_excel('mass_shooters.xlsx', sheet_name='Full Database', header=1)
mass_shooters.shape

(189, 153)

In [3]:
# Verify .py file functionality
mass_shooters_py = w.acquire_mass_shooters()
mass_shooters_py.shape

(189, 153)

<div style='background-color: orange'>
<a id="prepare"></a>
    <h1 style='text-align: center'>
        <b><i>
            Prepare
        </i></b></h1>
<li><a href='#TableOfContents'>Table of Contents</a></li>

<div style='background-color: orange'>
<a id="wrangle"></a>
    <h1 style='text-align: center'>
        <b><i>
            Wrangle
        </i></b></h1>
<li><a href='#TableOfContents'>Table of Contents</a></li>

<div style='background-color: orange'>
<a id="misc"></a>
    <h1 style='text-align: center'>
        <b><i>
            Miscellaneous
        </i></b></h1>
<li><a href='#TableOfContents'>Table of Contents</a></li>